# CDSE Basic Data Access
This notebook demonstrates the simplest way to authenticate with the Copernicus Data Space Ecosystem (CDSE), download a Sentinel-2 image, and render it.

In [ ]:
import os
import io
import requests
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [ ]:
# AUTHENTICATION
# Replace with your actual credentials or ensure environment variables are set
# Please make sure to not share a notebook with filled credentials!
CLIENT_ID = os.getenv("cdse_CLIENT_ID", "your_id_here")
CLIENT_SECRET = os.getenv("cdse_CLIENT_SECRET", "your_secret_here")
TOKEN_URL = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=TOKEN_URL, client_secret=CLIENT_SECRET, include_client_id=True)
print("Authenticated successfully.")

In [ ]:
# 1. Search for the latest cloud-free scene (Max 5% cloud)
search_url = "https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json"
search_params = {
    "box": "16.3,48.15,16.4,48.25",
    "startDate": "2024-04-01T00:00:00Z",
    "completionDate": "2024-06-01T00:00:00Z",
    "cloudCover": "[0,5]",
    "processingLevel": "S2MSI2A",
    "maxRecords": 1
}

search_resp = requests.get(search_url, params=search_params)
search_resp.raise_for_status()
features = search_resp.json().get('features', [])

if not features:
    raise Exception("No scenes found in this range. Try widening the dates.")

# Get the exact timestamp of the found scene
found_date = features[0]['properties']['startDate']
print(f"Found valid scene from: {found_date}")

In [ ]:

# Parse the found date and create a time window
# We remove the trailing 'Z' to make it compatible with fromisoformat
dt_str = found_date.replace('Z', '')
dt = datetime.fromisoformat(dt_str)

# Create a 1-hour window around the scene
time_from = (dt - timedelta(minutes=30)).strftime("%Y-%m-%dT%H:%M:%SZ")
time_to = (dt + timedelta(minutes=30)).strftime("%Y-%m-%dT%H:%M:%SZ")

print(f"Requesting time window: {time_from} to {time_to}")

# 2. Updated Payload with buffer
evalscript = """
//VERSION=3
function setup() {
    return {
        input: [{ bands: ["B04", "B03", "B02"], units: "DN" }],
        output: { bands: 3, sampleType: "UINT16" }
    };
}
function evaluatePixel(sample) {
    return [sample.B04, sample.B03, sample.B02];
}
"""

payload = {
    "input": {
        "bounds": {
            "bbox": [16.3, 48.15, 16.4, 48.25],
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"}
        },
        "data": [{
            "type": "sentinel-2-l2a",
            "dataFilter": {
                "timeRange": {
                    "from": time_from,
                    "to": time_to
                }
            }
        }]
    },
    "output": {
        "width": 512, 
        "height": 512,
        "responses": [{"identifier": "default", "format": {"type": "image/tiff"}}]
    },
    "evalscript": evalscript
}

response = oauth.post("https://sh.dataspace.copernicus.eu/api/v1/process", json=payload)
response.raise_for_status()
print("Data retrieved successfully!")

In [ ]:
import numpy as np

with rasterio.open(io.BytesIO(response.content)) as src:
    img = src.read().transpose(1, 2, 0)

# Check if we actually have data
print(f"Max pixel value: {img.max()}")
print(f"Mean pixel value: {img.mean()}")

# Scale: 3000 is a good baseline for S2; increase to 2000 for even brighter images
img_display = np.clip(img / 3000, 0, 1)

plt.figure(figsize=(5, 5))
plt.imshow(img_display)
plt.axis('off')
plt.show()